# Защита персональных данных клиентов

## 1. Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
import numpy.random
from sklearn.linear_model import LinearRegression

In [2]:
data = pd.read_csv('...')

In [3]:
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [5]:
data.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [6]:
data.duplicated().sum()

153

Удаляем дублированные строки:

In [7]:
data = data.drop_duplicates()

В нашем распоряжении файл с данными "insurance.csv". Данные организованы типами int64, float64. В файле имеется 5 столбцов признаков и 5000 строк (4847 после удаления дубликатов). Пропуски данных отсутствуют.

## 2. Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

- $E$ — единичная  матирица

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** При умножении признаков на обратимую матрицу качество линейной регрессии не изменится.

**Обоснование:**

Запишем формулу для предсказаний линейной регрессии после домножения на матрицу $P$:

$a_1 = XPw_1$, где 
$w_1$ — новая матрица коэффицинтов,
$a_1$ — новая матрица предсказаний.

Расчет $w_1$ соответственно примет вид:

$w_1 = ((XP)^TXP)^{-1}(XP)^Ty$

$w_1 = (P^TX^TXP)^{-1}P^TX^Ty$

$w_1 = (X^TXP)^{-1}(P^T)^{-1}P^TX^Ty$

$w_1 = P^{-1}(X^TX)^{-1}EX^Ty$

$w_1 = P^{-1}(X^TX)^{-1}X^Ty$

Подставим в формулу вычисления $a_1$ расчет $w_1$ и упростим выражение:

$a_1 = XPP^{-1}(X^TX)^{-1}X^Ty$

$a_1 = XE(X^TX)^{-1}X^Ty$

$a_1 = X(X^TX)^{-1}X^Ty$

Т.о., $a_1 = a$

Аналитически получено док.-во того, что новые предсказания будут идентичны первоначальным. Веса новой модели $w_1$ получают путем домножения $w$ на коэффицинт-матрицу $P^{-1}$.

## 3. Алгоритм преобразования

**Алгоритм**

Пусть $X$ — матрица признаков размерности $m$ x $(n+1)$, где $m$ — кол.-во объектов (записей), $n$ — кол.-во признаков (нулевой столбец заполнен единицами). Тогда необходимо взять такую матрицу P, чтобы её размерность была ($n+1$) x ($n+1$), а сама матрица была обратимой. Путем умножения матрицы $X$ на $P$ получаем новую матрицу $X_1$ с "зашифрованными" значениями признаков. При этом для восстановления изначальных значений необходимо матрицу $X_1$ умножить на $P^{-1}$.

**Обоснование**

Как было показано в п.2, при домножении матрицы признаков $X$ на обратимую матрицу $P$ предсказания модели не изменяются. Т.о., путем умножения $XP$ мы достигаем цели шифрования матрицы с персональными данными с возможностью дешифрования, а также обеспечиваем рабоспособность линейной регрессии (способность обучения и предсказания необходимых результатов).

## 4. Проверка алгоритма

Разделяем данные на обучающую и тренировочную выборки:

In [8]:
features = data.drop('Страховые выплаты', axis=1)

In [9]:
target = data['Страховые выплаты']

In [10]:
features_train, features_valid, target_train, target_valid = train_test_split(
                features, target, test_size=0.25, random_state=1234)

Добавляем нулевой столбец с единицами в переменные с признаками:

In [11]:
features_train = np.concatenate((np.ones((features_train.shape[0], 1)), features_train), axis=1)
features_train.shape

(3635, 5)

In [12]:
features_valid = np.concatenate((np.ones((features_valid.shape[0], 1)), features_valid), axis=1)
features_valid.shape

(1212, 5)

Генерируем матрицу P размерности ($n+1$) x ($n+1$), т.е. 5 х 5 со случайными целыми числами из диапазона 0...99:

In [13]:
P = np.random.randint(100,size=(5,5))

In [14]:
P

array([[72, 34, 36, 56, 19],
       [24, 11, 68,  4, 40],
       [ 3, 52, 44, 67, 97],
       [95, 93, 98, 85, 10],
       [33, 30, 45, 41,  4]])

Проверяем P на обратимость:

In [15]:
np.linalg.inv(P)

array([[ 0.01964478,  0.00515284, -0.00526382,  0.0036065 , -0.02620963],
       [-0.01594741, -0.00645433,  0.0046444 ,  0.03009407, -0.04756823],
       [-0.00964574,  0.01073546, -0.00316867, -0.00469164,  0.02703195],
       [ 0.00562112, -0.01185323,  0.00364269, -0.01972553,  0.05281075],
       [ 0.00843432,  0.00661828,  0.00690355, -0.00049142, -0.02242834]])

Создаем и обучаем модель на изначальных признаках, получаем предсказания, находим метрику R2:

In [16]:
model = LinearRegression()

In [17]:
model.fit(features_train, target_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [18]:
predictions = model.predict(features_valid)

In [19]:
R2 = r2_score(target_valid, predictions)
R2

0.421458478426583

Создаем модель на "зашифрованных" данных, обучаем, получаем предсказания, находим метрику R2:

In [20]:
features_train_1 = features_train @ P

In [21]:
features_valid_1 = features_valid @ P

In [22]:
model_1 = LinearRegression()

In [23]:
model_1.fit(features_train_1, target_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [24]:
predictions_1 = model_1.predict(features_valid_1)

In [25]:
R2_P = r2_score(target_valid, predictions_1)
R2_P

0.4214584784265373

In [26]:
R2 - R2_P

4.574118861455645e-14

Разница между полученными коэффициентами детерминации R2 незначительна и может появиться в процессе расчетов с плавающей точкой. Т.о., получно численное доказательство возможности преобразования матрицы признаков и построение модели линейной регрессии с идентичными значениями метрик, описанное в п.3.

## Чек-лист проверки

- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные загружены
- [x]  Выполнен шаг 2: получен ответ на вопрос об умножении матриц
    - [x]  Указан правильный вариант ответа
    - [x]  Вариант обоснован
- [x]  Выполнен шаг 3: предложен алгоритм преобразования
    - [x]  Алгоритм описан
    - [x]  Алгоритм обоснован
- [x]  Выполнен шаг 4: алгоритм проверен
    - [x]  Алгоритм реализован
    - [x]  Проведено сравнение качества моделей до и после преобразования